In [85]:
from transformers import pipeline, set_seed, Trainer
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForCausalLM
from datasets import load_dataset, DatasetDict

from torch.utils.data import Dataset, DataLoader

In [27]:
set_seed(123) # reproducibility


file_name = "../data/songs_data.txt"

In [103]:
class SongsDataset(Dataset):
    """Tokenize data when calling __getitem__"""
    
    def __init__(self, file_path, tokenizer):
        self.tokenizer = tokenizer
        self.data = []
        with open(file_name, "r") as f:
            song = ""
            for line in f:
                if line=="\n":
                    self.data.append({"text":song})
                    song = ""
                else:
                    song += line 
                    song += " "
                    
    #def __getitem__(self, i):
    #    inputs = self.tokenizer(self.data[i]["text"])
    #    return inputs
    
    def __len__(self):
        return len(self.data)

In [ ]:
tokenized_songs = tokenizer()

In [116]:
trainset = SongsDataset(file_name, tokenizer)
# trainset = trainset.map(lambda example: tokenizer(example, padding=True, truncation=True),
#     batched=True,
#     batch_size=16
# )
trainset = tokenizer(songs)#, padding=True, truncation=True)
train_dataloader = DataLoader(trainset, batch_size=1, collate_fn=lambda x: x )

In [105]:
#SongsDataset(file_name, tokenizer).data

In [ ]:
# trainer = Trainer(model=model,
#                   args=training_args,
#                   data_collator=data_collator,
#                   train_dataset=train_dataset,
#                   eval_dataset=eval_dataset,
#                   )

In [89]:
train_dataloader

In [118]:
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm


num_epochs = 3
num_training_steps = 3 * len(train_dataloader)
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

best_val_loss = float("inf")
progress_bar = tqdm(range(num_training_steps))
for epoch in range(num_epochs):
    # training
    model.train()
    for batch_i, batch in enumerate(train_dataloader):
        print(batch)
        # batch = ([text1, text2], [0, 1])

        output = model(**batch)
        
        optimizer.zero_grad()
        output.loss.backward()
        optimizer.step()
        lr_scheduler.step()
        progress_bar.update(1)
    
    # validation
    model.eval()
    for batch_i, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            output = model(**batch)
        loss += output.loss
    
    avg_val_loss = loss / len(eval_dataloader)
    print(f"Validation loss: {avg_val_loss}")
    if avg_val_loss < best_val_loss:
        print("Saving checkpoint!")
        best_val_loss = avg_val_loss
        print(f"best_val_loss = {best_val_loss}")
        #torch.save({
        #    'epoch': epoch,
        #    'model_state_dict': model.state_dict(),
        #    'optimizer_state_dict': optimizer.state_dict(),
        #    'val_loss': best_val_loss,
        #    },
        #    f"checkpoints/epoch_{epoch}.pt"
        #)  

  0%|          | 0/6 [00:00<?, ?it/s]

[Encoding(num_tokens=151, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]


TypeError: GPT2LMHeadModel object argument after ** must be a mapping, not list

In [61]:
songs = []
with open(file_name, "r") as f:
    song = ""
    for line in f:
        if line=="\n":
            songs.append(song)
            song = ""
        else:
            song += line 
            song += " "

In [83]:
model.training = True

In [84]:
model.train(songs)

ValueError: training mode is expected to be boolean

In [48]:
#model = AutoModelWithLMHead.from_pretrained('flax-community/papuGaPT2')
len(tokenizer(songs[0]).input_ids)


150

In [50]:
len(tokenizer(songs[0]).input_ids)

151

In [62]:
songs[0]

' Poprzez morza i przez oceany\n płynie statek wśród wspienionych fal\n a na statku chłopak zakochany \n zakochany w  morzu tyle lat \n Biała mewo, leć daleko stąd,\n leć daleko na ojczyzny ląd,\n poleć, powiedz, że marynarz chwat,\xa0\n morze kocha od dziecinnych lat\n  Nad okretem biała mewa leci \n nad podkładem gdzies się zerwał wiatr \n a nad statkiem czarne chmury wiszą\n a marynarz swą melodię gra \n  Na ojczystej ziemi pioska padła \n biała mewa też upadła w piach \n i o marynarzu opowiada \n co na morzy przezył tyle lat \n polec ...\n '

In [9]:
model = AutoModelForCausalLM.from_pretrained('flax-community/papuGaPT2')
tokenizer = AutoTokenizer.from_pretrained('flax-community/papuGaPT2')
generator = pipeline('text-generation', model='flax-community/papuGaPT2')

In [11]:
generator = pipeline('text-generation', model='flax-community/papuGaPT2')

In [13]:
generator("Najwiekszym polskim poeta byl")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Najwiekszym polskim poeta byl Julian Przyboczny, najdluzej poetkom znany w zaborach, a takze ich przyjaciele, ale tez i wielu, ktorzy zyja i pamietaja je z kart jego poezji. W tych czasach jednak'}]

In [16]:
model.__dict__.keys()

dict_keys(['training', '_parameters', '_buffers', '_non_persistent_buffers_set', '_backward_hooks', '_is_full_backward_hook', '_forward_hooks', '_forward_pre_hooks', '_state_dict_hooks', '_load_state_dict_pre_hooks', '_modules', 'config', 'name_or_path', 'model_parallel', 'device_map'])

In [23]:
method_list = [func for func in dir(model) if callable(getattr(model, func)) and not func.startswith("__")]

In [ ]:
model.train()